In [2]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [3]:
wine = load_wine()
X, y = wine.data, wine.target

In [4]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = pd.get_dummies(y).values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
def build_tf_model():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(y_train.shape[1], activation='softmax')
    ])
    model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

In [7]:
tf_start = time.time()
tf_model = build_tf_model()
tf_history = tf_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
tf_time = time.time() - tf_start

_, tf_acc = tf_model.evaluate(X_test, y_test, verbose=0)


c:\Users\ASLAM\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
class PyTorchModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, 64)
        self.layer2 = nn.Linear(64, output_dim)
        
    def forward(self, x):
        x = torch.relu(self.layer1(x))
        return torch.softmax(self.layer2(x), dim=1)


In [9]:

X_train_torch = torch.FloatTensor(X_train)
y_train_torch = torch.FloatTensor(y_train)
X_test_torch = torch.FloatTensor(X_test)
y_test_torch = torch.FloatTensor(y_test)


In [10]:

train_data = TensorDataset(X_train_torch, y_train_torch)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)


In [11]:

pt_model = PyTorchModel(X_train.shape[1], y_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pt_model.parameters())


In [12]:

pt_start = time.time()
for epoch in range(10):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = pt_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
pt_time = time.time() - pt_start


In [13]:

with torch.no_grad():
    pt_outputs = pt_model(X_test_torch)
    _, pt_preds = torch.max(pt_outputs, 1)
    _, y_true = torch.max(y_test_torch, 1)
    pt_acc = (pt_preds == y_true).float().mean().item()


In [14]:

results = pd.DataFrame({
    'Framework': ['TensorFlow', 'PyTorch'],
    'Training Time (s)': [tf_time, pt_time],
    'Test Accuracy': [tf_acc, pt_acc]
})

print("Framework Comparison Results:")
print(results.to_string(index=False))



Framework Comparison Results:
 Framework  Training Time (s)  Test Accuracy
TensorFlow           2.161998       0.972222
   PyTorch           0.132000       0.888889
